In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, '../') # import modules from ../

In [2]:
import pymysql
pymysql.install_as_MySQLdb()
import MySQLdb
import sys

MIN_TEMP=12
MAX_TEMP=30

# Setup MySQL
... and retrieve sensor locations

In [3]:
def setup_mysql(mysqlConfig:dict):

    try:
        db = MySQLdb.connect(**mysqlConfig)
        db.autocommit(True)
        dbc = db.cursor(pymysql.cursors.DictCursor)

    except Exception:
        print("Failed setting up database connection!", file=sys.stderr)
        raise

    return db, dbc

mysqlConfig={
    'db': 'buildax',
    'host': 'localhost',
    'passwd': 'fYFwecZKuUfvGE2e',
    'user': 'buildax'
}
db, dbc = setup_mysql(mysqlConfig)

In [4]:
dbc.execute("""
SELECT *, FROM_UNIXTIME(latest.latest) AS 'when' FROM buildax.metadata 
    JOIN (SELECT address, MAX(timestamp) as latest
            FROM buildax.buildax
            WHERE buildax.buildax.timestamp 
                BETWEEN UNIX_TIMESTAMP('2016-06-19 00:00:00') AND UNIX_TIMESTAMP('2016-06-26 00:00:00')
            GROUP BY address) AS latest 
        ON buildax.metadata.address = latest.address
    JOIN buildax.buildax
        ON buildax.buildax.address = latest.address
            AND buildax.buildax.timestamp = latest.latest
    WHERE buildax.metadata.BuildingID=0
        AND buildax.metadata.Floor=4;
""")
sensors=dbc.fetchall()
print(len(sensors))

sensors = [{
    'address': s['address'],
    'Latitude': s['Latitude'], 
    'Longitude': s['Longitude'],
    'Temp': s['Temp']
} for s in sensors]

dbc.close()
sensors[1]

13


{'Latitude': Decimal('54.980194'),
 'Longitude': Decimal('-1.609584'),
 'Temp': 21.1,
 'address': '4227C5D2'}

# Plot sensors on map

In [5]:
import cartopy
import cartopy.crs as ccrs
from cartopy.io.img_tiles import OSM
from cartopy._crs import Globe
import matplotlib.pyplot as plt
import numpy as np
import shapely
from shapely.geometry.point import Point

normalise = lambda x: (x-min(x))/(max(x)-min(x))

xpoints = [float(s['Longitude']) for s in sensors]
ypoints = [float(s['Latitude']) for s in sensors]
values = [float(s['Temp']) for s in sensors]
address = [s['address'] for s in sensors]

In [6]:
def extents(xpoints, ypoints, tolerance=0.0):
    return [min(xpoints) - tolerance, 
            max(xpoints) + tolerance, 
            min(ypoints) - tolerance, 
            max(ypoints) + tolerance]

extent = extents(xpoints,ypoints)

In [7]:
plt.rcParams["figure.figsize"] = (12,8)

projection = ccrs.PlateCarree()

# Find the 4 corners for the extent

borders = [-0.0001, 0.00005, -0.00005, 0.00005]
extents = [v + borders[i] for i,v in enumerate(extent)]

fig = plt.figure()
ax1 = fig.add_subplot(111, projection=projection)

ax1.set_extent(extents, projection)

for label, x, y in zip(address, xpoints, ypoints):
     ax1.annotate(
        label, 
        xy=(x, y),
        xytext=(-10, 25), 
        transform=projection._as_mpl_transform(ax1),
        textcoords='offset points', ha='right', va='top',
        bbox=dict(boxstyle='round,pad=0.5', fc='#ffeec9'), #, alpha=0.5),
        arrowprops=dict(arrowstyle='-', connectionstyle='angle,angleA=-90,angleB=180,rad=5'), zorder=0) #'arc3,rad=0'))


ax1.scatter(xpoints, ypoints, 
            c=normalise(np.array([float(s['Temp']) for s in sensors])),
            vmin=MIN_TEMP, vmax=MAX_TEMP,
            s=200, transform=projection._as_mpl_transform(ax1), 
            zorder=2)#, cmap=cm.coolwarm)


plt.tight_layout()
#plt.show()

# Save as an SVG
fig.patch.set_alpha(0)
ax1.background_patch.set_alpha(0)
fig.savefig('cover.svg', transparent=True)

plt.close()

# Heatmap
Aren't heatmaps great? Generate an extrapolated heatmap and save it as a png

In [18]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from scipy.interpolate import interp2d
from scipy.interpolate import griddata

normx = normalise(np.array(xpoints))
normy = normalise(np.array(ypoints))

grid_x, grid_y = np.mgrid[0:1:100j, 0:1:100j]
points = np.array(list(zip(normx, normy)))

In [9]:
# Create extents data for extrapolation

e = [np.argmax(normx), # index of max x
    np.argmin(normx), # index of min x
    np.argmax(normy), # index of max y
    np.argmin(normy)] # index of min y

top = 1.0
bot = 0.0
extentpoints = [[top, top], [bot, top], [bot, bot], [top, bot]]
extentvalues = [np.mean([values[e[0]], values[e[2]]]),
                np.mean([values[e[1]], values[e[2]]]),
                np.mean([values[e[0]], values[e[3]]]),
                np.mean([values[e[1]], values[e[3]]])]

values += extentvalues

points = np.concatenate((points, extentpoints))

grid_z2 = griddata(points, values, (grid_x, grid_y), method='cubic') #, fill_value=np.mean(values))

In [17]:
fig = plt.figure(figsize=(12,12))
ax1 = fig.add_subplot(111, aspect='equal')

m = ax1.imshow(grid_z2.T, 
               vmin=MIN_TEMP, vmax=MAX_TEMP,
               extent=(-.2,1.2,-.2,1.2), 
               origin='lower')

#fig.colorbar(m)
ax1.axis('off')

plt.tight_layout()
fig.savefig('heatmap.png')

#plt.show()
plt.close()

## Merge!
Time to put it all together

In [11]:
from shapely.ops import cascaded_union
from cartopy.io.shapereader import Reader

# Read geometry from shapefile and convert to matplotlib.path.Path via vertices
shprd = Reader("/Users/sam/Dropbox/PhD/BuildAX/89 Sandyford Road/GIS/89sandyford4F.shp")
polygon = list(shprd.geometries())[0]
polygon = cascaded_union(list(polygon.geoms))

verts = list(polygon.exterior.coords)
build_path = matplotlib.path.Path(verts)

In [12]:
matplotlib.rcParams['image.composite_image'] = False

fig = plt.figure()

ax1 = fig.add_subplot(111, projection=ccrs.PlateCarree())
ax1.set_extent(extents, ccrs.PlateCarree())

# OpenStreetMap layer background image
osm_tiles = OSM()
ax1.add_image(osm_tiles, 19, zorder=0)
# ax1.set_title("89 Sandyford Heatmap", x=0.1, y=0.9)

ax2 = fig.add_subplot(111, projection=ccrs.PlateCarree())
ax2.set_extent(extents, ccrs.PlateCarree())

# clip the image according to the logo_path. mpl v1.2.0 does not support
# the transform API that cartopy makes use of, so we have to convert the
# projection into a transform manually
transform = ccrs.PlateCarree()._as_mpl_transform(ax2)

im = plt.imshow(plt.imread('heatmap.png'),
                transform=ccrs.PlateCarree(), 
                extent=extents, zorder=1)
im.set_clip_path(build_path, transform=transform)

# add the path as a patch, drawing black outlines around the text
patch = matplotlib.patches.PathPatch(build_path, facecolor='none', edgecolor='black',
                                     transform=ccrs.PlateCarree())
ax2.add_patch(patch)

plt.tight_layout()
plt.savefig('background.svg')

# plt.show()
plt.close()

In [13]:
# Make a horizontal colourbar using the imshow axis from the heatmap plot above
fig = plt.figure()
ax1 = fig.add_subplot(111)
fig.colorbar(m, orientation='horizontal')
plt.tight_layout()
ax1.set_visible(False) 
fig.patch.set_alpha(0)
plt.savefig('colorbar.svg')
#plt.show()
plt.close()

In [14]:
import svgutils.compose as sc

building=sc.SVG("/Users/sam/Dropbox/PhD/BuildAX/89 Sandyford Road/maps/Floor4.svg")
building.rotate(-20)
building.skew(30, 0)
building.scale_xy(.7,.5)
building.move(20,324)

# here starts the assembling using svgutils 
sc.Figure("24cm", "16cm", 
    sc.Panel(
        sc.SVG("background.svg"),
        building,
        sc.SVG("cover.svg")),
    sc.Panel(
        sc.SVG('colorbar.svg').move(0, 40))
).save("compose.svg")


In [15]:
from IPython.display import SVG # /!\ note the 'SVG' function also in svgutils.compose
SVG('compose.svg')